In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

# Load and preprocess the data
path = Path("/kaggle/input/utkface-new/UTKFace/")
filenames = list(map(lambda x: x.name, path.glob('*.jpg')))
np.random.shuffle(filenames)

# Extract labels
age_labels, gender_labels, race_labels, image_path = [], [], [], []
for filename in filenames:
    image_path.append(filename)
    temp = filename.split('_')
    age_labels.append(temp[0])
    gender_labels.append(temp[1])
    race_labels.append(temp[2])

# Create DataFrame
df = pd.DataFrame()
df['image'], df['age'], df['gender'], df['race'] = image_path, age_labels, gender_labels, race_labels
df['gender'] = df['gender'].astype('int32')
df['age'] = df['age'].astype('int32')
df['race'] = df['race'].astype('int32')

# Train-Test Split
train, test = train_test_split(df, train_size=0.8, random_state=42)

# Image preprocessing
x_train = []
for file in train.image:
    img = load_img("/kaggle/input/utkface-new/UTKFace/" + file).convert('RGB')
    img = img.resize((224, 224))
    x_train.append(np.array(img))

x_train = np.array(x_train)

y_gender = np.array(train.gender)
y_age = np.array(train.age)
y_race = to_categorical(np.array(train.race), num_classes=5)

# Model building
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in conv_base.layers:
    layer.trainable = False

inputs = Input((224, 224, 3))
X = conv_base(inputs)
X = Flatten()(X)
X = Dense(256, activation='relu')(X)
X = Dropout(0.4)(X)

output_gender = Dense(1, activation='sigmoid', name='gender_output')(X)
output_age = Dense(1, activation='linear', name='age_output')(X)
output_race = Dense(5, activation='softmax', name='race_output')(X)

model = Model(inputs=inputs, outputs=[output_gender, output_age, output_race])

model.compile(optimizer='adam', 
              loss={'gender_output': 'binary_crossentropy',
                    'age_output': 'mean_squared_error',
                    'race_output': 'categorical_crossentropy'},
              metrics={'gender_output': 'accuracy', 
                       'age_output': 'mean_absolute_error',
                       'race_output': 'accuracy'})

# Checkpoint callback
checkpoint = ModelCheckpoint('best_model_by_race_output.h5', monitor='val_race_output_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history = model.fit(x_train, {'gender_output': y_gender, 'age_output': y_age, 'race_output': y_race}, epochs=20, batch_size=32, validation_split=0.1, callbacks=[checkpoint])
